In [1]:
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1543890669361_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [9]:
import os

datasets_path = os.path.join('datasets')

complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')


VBox()

In [11]:
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print "There are %s recommendations in the complete dataset" % (complete_ratings_data.count())

VBox()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 53886)
----------------------------------------
There are 27753444 recommendations in the complete dataset
Traceback (most recent call last):
  File "/usr/lib64/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 263, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 238, in poll
    if func():
  File "/usr/lib/spark/python/l

In [14]:
complete_ratings_data.take(3)

VBox()

[(1, 307, 3.5), (1, 481, 3.5), (1, 1091, 1.5)]

In [15]:
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')

complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
complete_movies_data.take(3)

VBox()

[(u'1', u'Toy Story (1995)'), (u'2', u'Jumanji (1995)'), (u'3', u'Grumpier Old Men (1995)')]

In [16]:
training_RDD, validation_RDD, test_RDD = complete_ratings_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

VBox()

In [17]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

VBox()

For rank 4 the RMSE is 0.8322223687
For rank 8 the RMSE is 0.818662418761
For rank 12 the RMSE is 0.820807934078
The best model was trained with rank 8

In [18]:
predictions.take(3)

VBox()

[((228384, 4898), 2.5090886633083818), ((228384, 2690), 3.3911831887885255), ((228384, 1730), 3.2263847727049453)]

In [19]:
rates_and_preds.take(3)

VBox()

[((52005, 2273), (2.0, 4.012166698769746)), ((67215, 83), (5.0, 4.084902528945154)), ((122015, 227), (3.0, 3.008871305978842))]

In [20]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

VBox()

For testing data the RMSE is 0.818791604071

In [21]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

VBox()

In [22]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,283,2), # New Jersey Drive (1995)
     (0,1,3), # Toy Story (1995)
     (0,16,3), # Casino (1995)
     (0,25,4), # Leaving Las Vegas (1995)
     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,1), # Flintstones, The (1994)
     (0,379,1), # Timecop (1994)
     (0,296,3), # Pulp Fiction (1994)
     (0,858,5) , # Godfather, The (1972)
     (0,50,4) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print 'New user ratings: %s' % new_user_ratings_RDD.take(10)

VBox()

New user ratings: [(0, 283, 2), (0, 1, 3), (0, 16, 3), (0, 25, 4), (0, 32, 4), (0, 335, 1), (0, 379, 1), (0, 296, 3), (0, 858, 5), (0, 50, 4)]

In [23]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

VBox()

In [24]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print "New model trained in %s seconds" % round(tt,3)

VBox()

New model trained in 97.435 seconds

In [25]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

VBox()

In [27]:
complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

VBox()

[(153600, ((2.483560624381123, u'Man van staal (1999)'), 2)), (122880, ((1.6660963574471876, u'I Want You (2012)'), 24)), (161760, ((2.0744321610956, u'Basta Poco (2015)'), 1))]

In [28]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

VBox()

In [29]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

VBox()

TOP recommended movies (with more than 25 reviews):
(u'Slaying the Badger', 3.966705157439656, 25)
(u'Magic & Bird: A Courtship of Rivals (2010)', 3.9580754817432635, 27)
(u'Mikra Anglia (2013)', 3.9410029556849295, 27)
(u'"Godfather', 3.923830960619915, 60904)
(u"Long Night's Journey Into Day (2000)", 3.886173660237716, 35)
(u'"Godfather: Part II', 3.8550242120448273, 38875)
(u'Cosmos', 3.744092866081248, 157)
(u'"Two Escobars', 3.743587541374063, 78)
(u'Pulp Fiction (1994)', 3.731949117108328, 92406)
(u'Small Potatoes - Who Killed the USFL? (2009)', 3.7110161774156656, 26)
(u'Voices from the List (2004)', 3.701612067494449, 1800)
(u'The Godfather Trilogy: 1972-1990 (1992)', 3.696807783857102, 421)
(u'Roots (2016)', 3.6779815597313883, 28)
(u'Frozen Planet (2011)', 3.667588669754866, 402)
(u"Schindler's List (1993)", 3.66587088193783, 71516)
(u'"Shawshank Redemption', 3.6654771227714313, 97999)
(u'Heimat - A Chronicle of Germany (Heimat - Eine deutsche Chronik) (1984)', 3.657693967084

In [30]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD.take(1)

VBox()

[Rating(user=0, product=27456, rating=3.0188843942477543)]